In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from scipy import misc
import os
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
import matplotlib.image as mpim
from PIL import Image
from IPython.display import display
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D, Conv2D
import keras
%matplotlib inline

Using TensorFlow backend.


In [2]:
im_size = 32
cl_number = 62

In [3]:
folder_list = ['Img/GoodImg/Bmp/', 'Img/BadImag/Bmp/']

Загрузка меток классов из датасета char74k

In [4]:
def get_74k_labels():
    with open('list_English_Img.m') as f:
        data = f.read()
        return data.split('];')[0].split('= [')[1].split(';\n')[:-1]

In [5]:
labels = get_74k_labels()

In [6]:
int_labels = [int(x) for x in labels]

Загрузка картинок из датасета char74k

In [7]:
images = np.empty((len(int_labels), 32, 32, 3))
i=0
for folder in folder_list:
    for sampl_fold in sorted(os.listdir(folder), key = lambda str: int(str.split('e0')[1])):
        for file in os.listdir(os.path.join(folder,sampl_fold)):
            image = misc.imread(os.path.join(folder, sampl_fold,file), mode = 'RGB')
            image = misc.imresize(image, (32, 32, 3))
            images[i] = image
            i += 1

One-hot-encoding для меток классов

In [8]:
binar_encoder = LabelBinarizer()
labels_b = binar_encoder.fit_transform(int_labels)
#y_test_b = binar_encoder.transform(y_test)

Создание и обучение модели на данных из датасета char74k

In [9]:
convnet = Sequential()
convnet.add(Conv2D(64,(10,10),activation='relu',
                   input_shape=(im_size,im_size, 3),
                   padding = 'same',
                   kernel_initializer='glorot_normal'))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(128,(7,7),activation='relu',padding = 'same',kernel_initializer='glorot_normal'))
convnet.add(Conv2D(128,(4,4),activation='relu',kernel_initializer='glorot_normal'))
convnet.add(MaxPooling2D())
convnet.add(Conv2D(256,(4,4),activation='relu',padding = 'same',kernel_initializer='glorot_normal'))
convnet.add(Flatten())
convnet.add(Dense(2048, activation='relu', kernel_initializer='glorot_normal'))
convnet.add(Dropout(0.5))
convnet.add(Dense(2048, activation='relu', kernel_initializer='glorot_normal'))
convnet.add(Dropout(0.5))
convnet.add(Dense(cl_number,  activation = 'softmax'))

In [10]:
convnet.compile(keras.optimizers.Adadelta(), keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [17]:
convnet.fit(images, labels_b, batch_size=32, epochs=50)

Epoch 1/50
12503/12503 [==============================] - 20s - loss: 0.0556 - acc: 0.9893    
Epoch 2/50
12503/12503 [==============================] - 20s - loss: 0.0609 - acc: 0.9888    
Epoch 3/50
12503/12503 [==============================] - 20s - loss: 0.0613 - acc: 0.9875    
Epoch 4/50
12503/12503 [==============================] - 20s - loss: 0.0669 - acc: 0.9884    
Epoch 5/50
12503/12503 [==============================] - 20s - loss: 0.0635 - acc: 0.9892    
Epoch 6/50
12503/12503 [==============================] - 20s - loss: 0.0796 - acc: 0.9879    
Epoch 7/50
12503/12503 [==============================] - 20s - loss: 0.0786 - acc: 0.9858    
Epoch 8/50
12503/12503 [==============================] - 20s - loss: 0.0671 - acc: 0.9882    
Epoch 9/50
12503/12503 [==============================] - 20s - loss: 0.0701 - acc: 0.9881    
Epoch 10/50
12503/12503 [==============================] - 20s - loss: 0.0789 - acc: 0.9865    
Epoch 11/50
12503/12503 [========================

Проверка модели на тренировочныз данныхю Видим что результат близок к 100%, и практически очевидно, что данные совпадают. Дальнейшая проверка методом 1nn это подтверждает.

In [12]:
train_labels = pd.read_csv('trainLabels.csv')
lb_coder = LabelEncoder()
lb_coder.fit(train_labels.Class.values)

LabelEncoder()

In [18]:
f_train = os.listdir('train')
images_train = np.zeros((len(f_train), im_size, im_size, 3))
for i, file in enumerate(sorted(f_train,key = lambda str: int(str.split('.')[0]))):
    #print(file)
    real_name = os.path.join('train', file)
    image = misc.imread(real_name,mode = 'RGB')
    image = misc.imresize(image,(im_size, im_size))
    #print(image.shape)
    images_train[i] = image
convnet.evaluate(images_train, binar_encoder.fit_transform(train_labels.Class.values))

6272/6283 [============================>.] - ETA: 0s

[0.033858203061927807, 0.9968168072576794]

Получаем предсказание для тестовой выборки

In [19]:
f_tests = os.listdir('test')
images_test = np.zeros((len(f_tests), im_size, im_size, 3))
for i, file in enumerate(sorted(f_tests,key = lambda str: int(str.split('.')[0]))):
    #print(file)
    real_name = os.path.join('test', file)
    image = misc.imread(real_name,mode = 'RGB')
    image = misc.imresize(image,(im_size, im_size))
    #print(image.shape)
    images_test[i] = image

In [26]:
res = convnet.predict(images_test)
test_csv = pd.read_csv('sampleSubmission.csv')
test_csv.drop('Class', inplace=True, axis = 1)
res_binar = binar_encoder.inverse_transform(res)

test_csv['Class'] = pd.Series(res_binar)
test_csv.to_csv('result74K.csv', index = False)

In [24]:
res_binar

array(['H', 'E', 'I', ..., 'R', 'N', 'M'],
      dtype='<U1')

In [102]:
knn = KNeighborsClassifier(1)

In [106]:
knn.fit(images.reshape((images.shape[0], im_size*im_size*3)), np.array(int_labels))
knn.score(images_test.reshape((images_test.shape[0], im_size*im_size*3)),train_labels.Class.values)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=1, p=2,
           weights='uniform')